# Setting Up SDK Artifacts

In [ ]:
import os
os.environ['SNPE_ROOT']="/local/mnt/workspace/aditya/qaisw-v2.15.1.230926150623_62883" #set up your snpe path here.
os.environ['RAW_FILE_FOLDER']="raw/"
os.environ['FOLDER_WITH_ARTIFACTS']="multiple"
os.environ['DLC32']="models/detr_resnet101_fp32.dlc"  # Use the path to your non-quantized dlc
os.environ['DLC8']="models/detr_resnet101_w8a8.dlc"              # Use the path to your Quantized dlc
os.environ['TARGET_INPUT_LIST']="list.txt"  # Use the name of the input file
os.environ['ONDEVICE_FOLDER']="aditya"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="2dce6316" #fill your device-id. Use command "adb devices" to get devices names. example :"e18d5d0"
os.environ['SNPE_TARGET_ARCH']="aarch64-android"
os.environ['SNPE_TARGET_STL']="libc++_shared.so"
os.environ['SNPE_TARGET_DSPARCH']="hexagon-v75"

In [ ]:
import math
import os
from PIL import Image
import requests
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import torch
import shutil
import numpy as np
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
torch.set_grad_enabled(False);
import os
import cv2
import numpy as np
from numpy import asarray
from PIL import Image
import glob
import torch.nn.functional as nnf
import subprocess
!pip3 install ipywidgets

## Getting the Dataset and Preparation

In [ ]:
# User needs to download the dataset of their choice, 
# and follow the pre/post processing steps prescribed for that dataset. 

### Pre-Processing Steps of DETR Model

In [ ]:
# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def plot_results(pil_img, prob, boxes,Image_count):
    fig=plt.figure(figsize=(8,8))
    ax1=fig.add_subplot(2,2,3)
    ax1.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=1))
        cl = p.argmax()
        text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=10,
                bbox=dict(alpha=0.5))
    plt.savefig(str(Image_count)+".jpg")
    if Image_count%2==0:
        shutil.move(str(Image_count)+".jpg","output/CPU")
    else:
        shutil.move(str(Image_count)+".jpg","output/DSP")
    plt.show()

### Steps to create raw images

In [ ]:
name="raw"
os.system('mkdir ' + name)
def detect(imgfile,i):
    #getting the actual image
    origimg = Image.open(imgfile)
    #Transforming the image
    img = transform(origimg).unsqueeze(0)

    img= nnf.interpolate(img, size=(800, 1066), mode='bicubic', align_corners=False)
    
    img_to_save=img.numpy().transpose(0,2,3,1).astype(np.float32)
    
    img_to_save.tofile("raw/"+filenames[i].split(".")[0]+".raw")
    
filenames = os.listdir("val2017")
for i in range(0,len(filenames)):
    if "jpg" in filenames[i].lower():
        detect("val2017/"+filenames[i],i)

In [ ]:
%%bash
find ./raw -name *.raw > list.txt

### Getting the Model

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
python generateModels.py

## Creating Bin and Lib Folder On Device

In [ ]:
%%bash
#source throughput.sh >>dump.txt
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/dsp/lib"

# Pusing All Bin and Lib Files on to Device

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/$SNPE_TARGET_STL /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/*.so /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib
$DEVICE_SHELL push $SNPE_ROOT/lib/hexagon-v75/unsigned/*.so /data/local/tmp/snpeexample/dsp/lib
$DEVICE_SHELL push $SNPE_ROOT/bin/$SNPE_TARGET_ARCH/snpe-net-run /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin

# Pushing Artifacts onto Device

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $RAW_FILE_FOLDER /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $TARGET_INPUT_LIST /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $DLC32 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $DLC8 /data/local/tmp/$ONDEVICE_FOLDER

# Inferencing 8-bit DLC onto DSP Runtime

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export SNPE_TARGET_ARCH=aarch64-android
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib
export PATH=$PATH:/data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin
export ADSP_LIBRARY_PATH="/data/local/tmp/snpeexample/dsp/lib;/system/lib/rfsa/adsp;/system/vendor/lib/rfsa/adsp;/dsp"
export OUTPUT_FOLDER=OUTPUT_8b_DSP
export DLC8=detr_resnet101_w8a8.dlc
export ONDEVICE_FOLDER="aditya"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $DLC8 --input_list list.txt  --set_output_tensors 5863,5862 --output_dir=OUTPUT_8b_DSP --use_dsp

# Inferencing 32-bit DLC onto CPU Runtime

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export SNPE_TARGET_ARCH=aarch64-android
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib
export PATH=$PATH:/data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin
export OUTPUT_FOLDER=OUTPUT_32b_CPU
export DLC32=detr_resnet101_fp32.dlc
export ONDEVICE_FOLDER="aditya"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $DLC32 --input_list list.txt --set_unconsumed_as_output --output_dir=OUTPUT_32b_CPU

# Pulling output folder generated on different Precision and Cores

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_8b_DSP OUTPUT_8b_DSP
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_32b_CPU OUTPUT_32b_CPU

## Post Processing the Inferenced data

In [ ]:
# Sample list of classes in a dataset
CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

In [ ]:
import matplotlib.pyplot as plt
Image_Paths=[]

with open('list.txt', 'r') as f:
    for line in f:
        Image_Paths.append(line.strip().split("/")[-1].split(".")[0])


count=Image_count=0
if os.path.exists("output")==False:
    os.mkdir("output")
if os.path.exists("output/CPU")==False:
    os.mkdir("output/CPU")
if os.path.exists("output/DSP")==False:
    os.mkdir("output/DSP")
for image in Image_Paths:
    image_path = 'val2017/'+image+".jpg"
    im = Image.open(image_path)
    file1 = 'OUTPUT_32b_CPU/Result_' + str(count) + '/5862.raw'
    file2 = 'OUTPUT_32b_CPU/Result_' + str(count) + '/5863.raw'
    file3 = 'OUTPUT_8b_DSP/Result_' + str(count) + '/5862.raw'
    file4 = 'OUTPUT_8b_DSP/Result_' + str(count) + '/5863.raw'
    a=np.fromfile(file1,np.float32)
    a=a.reshape(1,100,92)
    tensor_a = torch.from_numpy(a)
    b=np.fromfile(file2,np.float32)
    b=b.reshape(1,100,4)
    tensor_b = torch.from_numpy(b)

    c=np.fromfile(file3,np.float32)
    c=c.reshape(1,100,92)
    tensor_c = torch.from_numpy(c)
    d=np.fromfile(file4,np.float32)
    d=d.reshape(1,100,4)
    tensor_d = torch.from_numpy(d)


    
    probas = tensor_a.softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > 0.9
    bboxes_scaled = rescale_bboxes(tensor_b[0, keep], im.size)
    print("CPU FP32 Inference Result")
    plot_results(im, probas[keep], bboxes_scaled,Image_count)
    Image_count=Image_count+1

    probas = tensor_c.softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > 0.9
    bboxes_scaled = rescale_bboxes(tensor_d[0, keep], im.size)
    print("DSP INT8 Inference Result")
    plot_results(im, probas[keep], bboxes_scaled,Image_count)
    Image_count=Image_count+1
    count=count+1